# Практична робота 2: булевий пошук

*Done by Yuliia Hetman, SHI-2*

## Вхідні текстові файли


Для цього завдання я вирішила обрати твори українських класиків:
1.  -rw-rw-r-- 1 yhetman  41K  Intermezzo.txt
2.  -rw-rw-r-- 1 yhetman 555K  Захар_Беркут.txt
3.  -rw-rw-r-- 1 yhetman 990K  Земля.txt
4.  -rw-rw-r-- 1 yhetman  51K  Зів'яле_листя.txt
5.  -rw-rw-r-- 1 yhetman 472K  Кайдашева_сім'я.txt
6.  -rw-rw-r-- 1 yhetman  33K  Камінний_хрест.txt
7.  -rw-rw-r-- 1 yhetman 167K  Климко.txt
8.  -rw-rw-r-- 1 yhetman 937K  Кобзар.txt
9.  -rw-rw-r-- 1 yhetman 153K  Лісова_пісня.txt
10. -rw-rw-r-- 1 yhetman 141K  Мартин_Боруля.txt
11. -rw-rw-r-- 1 yhetman 261K  Маруся_Чурай.txt
12. -rw-rw-r-- 1 yhetman 173K  Мина_Мазайло.txt
13. -rw-rw-r-- 1 yhetman 914K  Місто.txt
14. -rw-rw-r-- 1 yhetman 834K  Тигролови.txt
15. -rw-rw-r-- 1 yhetman 170K  Тіні_забутих_предків.txt
16. -rw-rw-r-- 1 yhetman 1,2M  Хіба_ревуть_воли_як_ясла_повні.txt
17. -rw-rw-r-- 1 yhetman 584K  Чорна_Рада.txt
18. -rw-rw-r-- 1 yhetman  50K  Я_романтика.txt


При створені словника мною були знехтувані пунктуаційні та інші розділові знаки (крім апострофа). Також були прибрані всі цифри, що здебільшого використовувались для позначення дат. Зайві білі знаки та табуляція були також вирізані.

## Reading files

In [1]:
%%time
import re, os, json
from string import digits
import pandas as pd

docIDs = list(range(0, 18))
def delete_sings_digits(content):
    punc = '''!()-[]{};:"\,«»<>./?@#$%—…^&*_~'''
    for ele in content:
        if ele in punc:
            content = content.replace(ele, "")
    table = str.maketrans('', '', digits)
    content = content.translate(table)
    return content


def create_vocabulary(content):    
    raw_content = delete_sings_digits(content)
    vocab = [i.lower() for i in raw_content.split() if i != '']
    return vocab


def read_books(dirname='../books/'):
    books = os.listdir(dirname)
    vocabulary = list()
    dictionary = dict()
    memory = 0
    for book in books:
        with open(dirname + book, 'r', encoding='utf-8') as file:
            text = create_vocabulary(file.read())
            dictionary[book.replace('.txt', '')] = text
            vocabulary += text
        memory += os.stat(dirname + book).st_size
    return memory, vocabulary, dictionary


memory, vocabulary, dictionary = read_books()
print(f'Total number of words : {len(vocabulary)}')
print(f'Total number of tokens : {len(set(vocabulary))}')

Total number of words : 709181
Total number of tokens : 94971
CPU times: user 1min 4s, sys: 340 ms, total: 1min 4s
Wall time: 1min 4s


## Function to create Incidence Matrix

In [2]:
%%time

def create_incidence_matrix(dictionary):
    matrix = list()
    for book, words in dictionary.items():
        uni_words = list(set(words))
        column = dict()
        for word in uni_words: 
            column[word] = 1
        other = list(set(vocabulary) - set(uni_words))
        for word in other:
            column[word] = 0
        matrix.append(column)
    matrix = pd.DataFrame(matrix, index=list(dictionary.keys()), columns=list(set(vocabulary)))
    return matrix.transpose()

matrix = create_incidence_matrix(dictionary)

CPU times: user 4.75 s, sys: 164 ms, total: 4.92 s
Wall time: 4.92 s


In [3]:
matrix.head()

,Тигролови,Захар_Беркут,Камінний_хрест,Intermezzo,Хіба_ревуть_воли_як_ясла_повні,Чорна_Рада,Я_романтика,Земля,Кобзар,Лісова_пісня,Місто,Маруся_Чурай,Климко,Зів'яле_листя,Мина_Мазайло,Тіні_забутих_предків,Мартин_Боруля,Кайдашева_сім'я
зароблені,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
вкрадену,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
дівся,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,1
вражене,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
освічувало,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1


## Function to create Inverted Indexes

In [85]:
%%time

def create_invert_index(vocabulary, dictionary):
    uniq = list(set(vocabulary))
    voc = dict(zip(uniq, range(len(uniq))))
    freq = [0] * len(voc)
    docIDs = dict(zip(uniq, [-1] * len(uniq)))

    for j, book in enumerate(dictionary):
        uni_words = list(set(dictionary[book]))
        for word in uni_words:
            freq[voc[word]] += 1
            if docIDs[word] != -1:
                docIDs[word] += [j]
            else:
                docIDs[word] = [j]
    invert_indx = dict()
    for word in list(voc.keys()):
        invert_indx[word] = dict()
        invert_indx[word]['frequency'] = freq[voc[word]]
        invert_indx[word]['docIDs'] = docIDs[word]
    return invert_indx

invert_indx = create_invert_index(vocabulary, dictionary)

CPU times: user 681 ms, sys: 8 ms, total: 689 ms
Wall time: 686 ms


In [84]:
%%time

voc = dict(zip(list(set(vocabulary)), range(len(list(set(vocabulary))))))
freq = [0] * len(voc)
docIDs = dict(zip(list(set(vocabulary)), [-1] * len(list(set(vocabulary)))))

for j, book in enumerate(dictionary):
    uni_words = list(set(dictionary[book]))
    for word in uni_words:
        freq[voc[word]] += 1
        if docIDs[word] != -1:
            docIDs[word] += [j]
        else:
            docIDs[word] = [j]
            
# invert_indx = dict()#zip(list(set(vocabulary)), range(len(list(set(vocabulary))))))
for word in list(voc.keys()):
    invert_indx[word] = dict()
    invert_indx[word]['frequency'] = freq[voc[word]]
    invert_indx[word]['docIDs'] = docIDs[word]
    
#             invert_indx[word]['docIDs'] += [i]
    
    #         if word not in used_w:
#             invert_indx[word] = {'frequency': 1,
#                 'docIDs': [i]}
#             used_w.append(word)
#         else:
#             invert_indx[word]['frequency'] += 1
#             invert_indx[word]['docIDs'] += [i]
invert_indx

{'зароблені': {'docIDs': [4, 7], 'frequency': 2},
 'вкрадену': {'docIDs': [7], 'frequency': 1},
 'дівся': {'docIDs': [4, 5, 7, 8, 17], 'frequency': 5},
 'вражене': {'docIDs': [11], 'frequency': 1},
 'освічувало': {'docIDs': [4, 7, 17], 'frequency': 3},
 'інквізицією': {'docIDs': [0], 'frequency': 1},
 'дзеленчали': {'docIDs': [0], 'frequency': 1},
 'похилились': {'docIDs': [8, 17], 'frequency': 2},
 'втрати': {'docIDs': [0, 4, 7, 10, 13], 'frequency': 5},
 'пошпурила': {'docIDs': [0], 'frequency': 1},
 "закам'янів": {'docIDs': [4, 7], 'frequency': 2},
 'аркуш': {'docIDs': [7, 8, 10], 'frequency': 3},
 'звалось': {'docIDs': [0, 8], 'frequency': 2},
 'феодосія': {'docIDs': [17], 'frequency': 1},
 'перемигнув': {'docIDs': [1], 'frequency': 1},
 'множились': {'docIDs': [11], 'frequency': 1},
 "п'яніший": {'docIDs': [4], 'frequency': 1},
 'зачують': {'docIDs': [5], 'frequency': 1},
 "одв'язував": {'docIDs': [17], 'frequency': 1},
 'тройко': {'docIDs': [4], 'frequency': 1},
 'непотішнії': {'

In [77]:
word

'зароблені'

In [76]:
freq[voc[word]]

2

In [49]:
voc = dict(zip(list(set(vocabulary)), range(len(list(set(vocabulary))))))
# keys, values = voc.iteritems()
voc

{'зароблені': 0,
 'вкрадену': 1,
 'дівся': 2,
 'вражене': 3,
 'освічувало': 4,
 'інквізицією': 5,
 'дзеленчали': 6,
 'похилились': 7,
 'втрати': 8,
 'пошпурила': 9,
 "закам'янів": 10,
 'аркуш': 11,
 'звалось': 12,
 'феодосія': 13,
 'перемигнув': 14,
 'множились': 15,
 "п'яніший": 16,
 'зачують': 17,
 "одв'язував": 18,
 'тройко': 19,
 'непотішнії': 20,
 'диви': 21,
 'шинку': 22,
 'пограло': 23,
 'дражнись': 24,
 'причащалась': 25,
 'помчала': 26,
 'з’їсть': 27,
 'намовляли': 28,
 'зневагою': 29,
 'красне': 30,
 'громад': 31,
 'червоноруського': 32,
 'спустився': 33,
 'безславному': 34,
 'неохайного': 35,
 'зглядували': 36,
 'щасливцям': 37,
 'збіглися': 38,
 'шведови': 39,
 'непогоды': 40,
 'знівечив': 41,
 "п'ятиповерховий": 42,
 'промовила': 43,
 'покірливі': 44,
 'ліжку': 45,
 'сподіваної': 46,
 'чую”': 47,
 'глаголю': 48,
 'знеможений': 49,
 'службу': 50,
 'переможцею': 51,
 'іспитувати': 52,
 'граційна': 53,
 'спочинувши': 54,
 'приблуди': 55,
 'арена': 56,
 'порився': 57,
 'шусть'

In [5]:
invert_indx

{'зважав': {'docIDs': [0, 1, 5, 7, 10], 'frequency': 5},
 'виправдуючись': {'docIDs': [0], 'frequency': 1},
 'ластівко': {'docIDs': [0], 'frequency': 1},
 'телеграмаблискавка': {'docIDs': [0], 'frequency': 1},
 'йолопи': {'docIDs': [0], 'frequency': 1},
 'збивши': {'docIDs': [0], 'frequency': 1},
 'дзеленчали': {'docIDs': [0], 'frequency': 1},
 'краса': {'docIDs': [0, 4, 5, 8, 9, 11, 13, 14, 17], 'frequency': 9},
 'інквізицією': {'docIDs': [0], 'frequency': 1},
 'втомила': {'docIDs': [0], 'frequency': 1},
 'тамтешній': {'docIDs': [0], 'frequency': 1},
 'годинник': {'docIDs': [0, 6, 10, 13], 'frequency': 4},
 'передусім': {'docIDs': [0, 7, 10], 'frequency': 3},
 'втрати': {'docIDs': [0, 4, 7, 10, 13], 'frequency': 5},
 'цвіло': {'docIDs': [0, 8], 'frequency': 2},
 'затишок': {'docIDs': [0, 10], 'frequency': 2},
 'пошпурила': {'docIDs': [0], 'frequency': 1},
 'помовчавши': {'docIDs': [0, 4, 5, 7, 9, 11], 'frequency': 6},
 'звалось': {'docIDs': [0, 8], 'frequency': 2},
 'вгорі': {'docIDs'

In [6]:
def AND(arr1, arr2):
    result = len(arr1) * [0]
    for i in range(len(arr1)):
        result[i] = 1 if arr1[i] == 1 and arr2[i] == 1 else 0
    return result

In [7]:
def OR(arr1, arr2):
    result = len(arr1) * [0]
    for i in range(len(arr1)):
        result[i] = 1 if arr1[i] > 0 or arr2[i] > 0 else 0
    return result

In [8]:
def NOT(word):
    return [abs(i - 1) for i in word]

In [9]:
def AND_ind(word1, word2):
    return list(set(word1) & set(word2))

In [10]:
def OR_ind(word1, word2):
    return list(set(word1) | set(word2))

In [11]:
def NOT_ind(word, docIDs=docIDs):
    return [i for i in docIDs if i not in word]           

### Save incidence matrix to CSV-file

In [12]:
sizes = [0, 0]
matrix.to_csv('incidence_matrix.csv', index = True, header = True)
size = os.stat('incidence_matrix.csv').st_size
sizes[0] = {'incidence matrix': str(round((size/ (1024 * 1024)), 1)) + " MB"}
print(f'CSV-file size with incidence matrix is: {sizes[0]["incidence matrix"]}')

CSV-file size with incidence matrix is: 4.8 MB


### Save inverted indexes to CSV-file

In [13]:
pd.DataFrame(invert_indx).transpose().to_csv('invert_indexes.csv', index = True, header = True)
size = os.stat('invert_indexes.csv').st_size
sizes[0]['inverted indexes'] = str(round((size/ (1024 * 1024)), 1)) + " MB"
print(f'CSV-file size with inverted indexes is: {sizes[0]["inverted indexes"]}')

CSV-file size with inverted indexes is: 2.4 MB


### Save incidence matrix to JSON-file

In [14]:
with open('incidence_matrix.json', 'w') as file:
    json.dump(matrix.to_dict(), file, ensure_ascii=False)

size = os.stat('incidence_matrix.json').st_size
sizes[1] = {'incidence matrix': str(round((size/ (1024 * 1024)), 1)) + " MB"}
print(f'JSON-file size with incidence matrix is: {sizes[1]["incidence matrix"]}')

JSON-file size with incidence matrix is: 37.3 MB


### Save inverted indexes to JSON-file

In [15]:
with open('invert_indexes.json', 'w') as file:
    json.dump(invert_indx, file, ensure_ascii=False)
    
size = os.stat('invert_indexes.json').st_size
sizes[1]["inverted indexes"] = str(round((size/ (1024 * 1024)), 1)) + " MB"
print(f'JSON-file size with inverted indexes is: {sizes[1]["inverted indexes"]}')

JSON-file size with inverted indexes is: 5.1 MB


In [16]:
compare_size = pd.DataFrame(sizes, columns=['incidence matrix', 'inverted indexes'], index = ['csv', 'json'])
compare_size

,incidence matrix,inverted indexes
csv,4.8 MB,2.4 MB
json,37.3 MB,5.1 MB


## Check and compare two data structures

In [17]:
invert_indx['вічний']

{'docIDs': [1, 8, 10, 11, 15], 'frequency': 5}

In [18]:
matrix.loc['вічний'].values

array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0])

In [19]:
invert_indx['сказала']

{'docIDs': [0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17],
 'frequency': 15}

In [20]:
matrix.loc['сказала'].values

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

### Compare AND operation

In [21]:
%%time
AND(matrix.loc['вічний'].values, matrix.loc['сказала'].values)

CPU times: user 915 µs, sys: 2 µs, total: 917 µs
Wall time: 941 µs


[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0]

In [22]:
%%time
AND_ind(invert_indx['вічний']['docIDs'], invert_indx['сказала']['docIDs'])

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 19.3 µs


[1, 8, 10, 11, 15]

### Compare OR operation

In [23]:
%%time
OR(matrix.loc['вічний'].values, matrix.loc['сказала'].values)

CPU times: user 709 µs, sys: 2 µs, total: 711 µs
Wall time: 719 µs


[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]

In [24]:
%%time
OR_ind(invert_indx['вічний']['docIDs'], invert_indx['сказала']['docIDs'])

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 16.2 µs


[0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17]

### Compare NOT operation

In [25]:
%%time
NOT(matrix.loc['вічний'].values)

CPU times: user 351 µs, sys: 1e+03 ns, total: 352 µs
Wall time: 359 µs


[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1]

In [26]:
%%time
NOT_ind(invert_indx['вічний']['docIDs'])

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 13.8 µs


[0, 2, 3, 4, 5, 6, 7, 9, 12, 13, 14, 16, 17]

### Compare compicated operations

In [27]:
%%time
NOT(OR(AND(matrix.loc['вічний'].values, matrix.loc['тобі'].values), matrix.loc['зоріє'].values))

CPU times: user 664 µs, sys: 0 ns, total: 664 µs
Wall time: 670 µs


[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1]

In [28]:
%%time
NOT_ind(OR_ind(AND_ind(invert_indx['вічний']['docIDs'], invert_indx['тобі']['docIDs']), invert_indx['зоріє']['docIDs']))

CPU times: user 23 µs, sys: 0 ns, total: 23 µs
Wall time: 26.2 µs


[2, 3, 4, 5, 6, 7, 9, 12, 13, 14, 16, 17]